In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [2]:
def gerar_sopa(pg = 1):
    
    url = ('https://participemais.prefeitura.sp.gov.br/legislation/processes/116/'
    'proposals?page={pg}&random_seed=225243')
    url = url.format(pg = pg)
    
    with requests.get(url) as r:
        assert r.status_code == 200
        html = r.text
        
    sopa = BeautifulSoup(html)
    
    return sopa

In [3]:
def check_ultima_pag(sopa):
    
    pagination = sopa.find('div', {'class' : 'pagination-centered'})
    ultima = pagination.find_all('li')[-1].text.strip().lower()
    if ultima == 'último':
        return False
    return True

In [4]:
def paineis_propostas(sopa):
    
    div_propostas = sopa.find('div', {'class' : 'legislation-proposals'})
    paineis = div_propostas.find_all('div', {'class' : 'proposal clear'})
    
    return paineis

In [5]:
def autor(painel):
    
    autor = painel.find('span', {'class' : 'author'})
    
    return autor.text.strip()

def titulo(painel):
    
    return painel.find('h3').text.strip()

def link_proposta(painel):
    
    return painel.find('h3').find('a')['href']


#talvez dê para melhorar essa, mas seguimos
def data_proposta(painel):
    
    info_header = painel.find('p', {'class' : 'proposal-info'})
    info_header = info_header.text.split('\n')
    
    return info_header[4].strip()

def qt_apoios(painel):
    
    qt = painel.find('span', {'class' : 'total-votes center'})
    qt = qt.text.strip()
    if qt.lower() == 'nenhum apoio':
        return 0
    else:
        return int(''.join([char for char in qt if char.isdigit()]))
    
def temas(painel):
    
    temas = painel.find('ul', {'class' : 'no-bullet tags'})
    if not temas:
        return ''
    else:
        temas = temas.find_all('a')
        return '; '.join(tema.text for tema in temas)
    
def texto_proposta(painel):
    
    text = painel.find('div', {'class' : 'proposal-description'})
    
    return text.text.strip()
    

In [6]:
pg_num = 1
ultima_pg = False

todos_dados = []
while not ultima_pg:
    print(pg_num)
    sopa = gerar_sopa(pg_num)
    ultima_pg = check_ultima_pag(sopa)
    pg_num +=1
    
    paineis = paineis_propostas(sopa)
    
    for painel in paineis:
        
        dados = {
            'autor' : autor(painel),
            'link_proposta' : link_proposta(painel),
            'data_submissao' : data_proposta(painel),
            'qtd_apoios' : qt_apoios(painel),
            'temas' : temas(painel),
            'conteudo' : texto_proposta(painel)
        }
        
        todos_dados.append(dados)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


In [7]:
df = pd.DataFrame(todos_dados)

In [8]:
path_dados = '../Dados_site/'

In [9]:
hoje = datetime.datetime.today()
nom_arquivo = f'propostas_pdm_{hoje.day}_{hoje.month}.xlsx'

In [10]:
nom_arquivo

'propostas_pdm_14_5.xlsx'

In [11]:
df.to_excel(path_dados+nom_arquivo)

In [12]:
len(df)

864